In [1]:
ls ../../../GitHub/robotcar-dataset-sdk/

LICENSE  README.md  extrinsics/  matlab/  models/  python/  tags/


In [19]:
sdk_path = "/home/ios/GitHub/robotcar-dataset-sdk"
oxford_path = "/home/ios/data2/3d-data/oxford"

import sys
sys.path.append(sdk_path + "/python")
!ls {oxford_path}
!ls {extrinsics_dir}
!cat {extrinsics_dir}/mono_left.txt

iss_volume.zip	iss_zip  sift_patch_1.zip  sift_patch_2.zip  sift_zip
ins.txt        lms_rear.txt   mono_right.txt  velodyne_left.txt
ldmrs.txt      mono_left.txt  radar.txt       velodyne_right.txt
lms_front.txt  mono_rear.txt  stereo.txt
-0.0905 1.6375 0.2803 0.2079 -0.2339 1.2321


In [3]:
import argparse
from argparse import RawTextHelpFormatter
import os
from velodyne import load_velodyne_raw, load_velodyne_binary, velodyne_raw_to_pointcloud
import numpy as np
import cv2
from matplotlib.cm import get_cmap
from scipy import interpolate
import open3d
from transform import build_se3_transform
import meshplot

INFO - 2021-09-10 07:40:55,289 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2021-09-10 07:40:55,290 - utils - NumExpr defaulting to 8 threads.


In [4]:
im_dir_left = oxford_path + "/mono_left"
im_dir_right = oxford_path + "/mono_right"
poses_file = oxford_path + "/gps/ins.csv"
extrinsics_dir = sdk_path + "/extrinsics"
models = sdk_path + "/models"
lidar_dir = oxford_path + "/ldmrs"
!ls {extrinsics_dir}

ins.txt        lms_rear.txt   mono_right.txt  velodyne_left.txt
ldmrs.txt      mono_left.txt  radar.txt       velodyne_right.txt
lms_front.txt  mono_rear.txt  stereo.txt


In [5]:
import os
import re
import numpy as np

from transform import build_se3_transform
from interpolate_poses import interpolate_vo_poses, interpolate_ins_poses
from velodyne import load_velodyne_raw, load_velodyne_binary, velodyne_raw_to_pointcloud

def build_pointcloud(lidar_dir, poses_file, extrinsics_dir, start_time, end_time, origin_time=-1):
    """Builds a pointcloud by combining multiple LIDAR scans with odometry information.
    Args:
        lidar_dir (str): Directory containing LIDAR scans.
        poses_file (str): Path to a file containing pose information. Can be VO or INS data.
        extrinsics_dir (str): Directory containing extrinsic calibrations.
        start_time (int): UNIX timestamp of the start of the window over which to build the pointcloud.
        end_time (int): UNIX timestamp of the end of the window over which to build the pointcloud.
        origin_time (int): UNIX timestamp of origin frame. Pointcloud coordinates are relative to this frame.
    Returns:
        numpy.ndarray: 3xn array of (x, y, z) coordinates of pointcloud
        numpy.array: array of n reflectance values or None if no reflectance values are recorded (LDMRS)
    Raises:
        ValueError: if specified window doesn't contain any laser scans.
        IOError: if scan files are not found.
    """
    if origin_time < 0:
        origin_time = start_time

    lidar = re.search('(lms_front|lms_rear|ldmrs|velodyne_left|velodyne_right)', lidar_dir).group(0)
    timestamps_path = os.path.join(lidar_dir, os.pardir, lidar + '.timestamps')

    timestamps = []
    
    with open(timestamps_path) as timestamps_file:
        for line in timestamps_file:
            timestamp = int(line.split(' ')[0])
            if start_time <= timestamp <= end_time:
                timestamps.append(timestamp)

    print("Processing" , len(timestamps) , " scans.")
    if len(timestamps) == 0:
        raise ValueError("No LIDAR data in the given time bracket.")

    with open(os.path.join(extrinsics_dir, lidar + '.txt')) as extrinsics_file:
        extrinsics = next(extrinsics_file)
    G_posesource_laser = build_se3_transform([float(x) for x in extrinsics.split(' ')])

    poses_type = re.search('(vo|ins|rtk)\.csv', poses_file).group(1)

    if poses_type in ['ins', 'rtk']:
        with open(os.path.join(extrinsics_dir, 'ins.txt')) as extrinsics_file:
            extrinsics = next(extrinsics_file)
            G_posesource_laser = np.linalg.solve(build_se3_transform([float(x) for x in extrinsics.split(' ')]),
                                                 G_posesource_laser)

        poses = interpolate_ins_poses(poses_file, timestamps, origin_time, use_rtk=(poses_type == 'rtk'))
    else:
        # sensor is VO, which is located at the main vehicle frame
        poses = interpolate_vo_poses(poses_file, timestamps, origin_time)

    pointcloud = np.array([[0], [0], [0], [0]])
    if lidar == 'ldmrs':
        reflectance = None
    else:
        reflectance = np.empty((0))

    for i in range(0, len(poses)):
        scan_path = os.path.join(lidar_dir, str(timestamps[i]) + '.bin')
        if "velodyne" not in lidar:
            if not os.path.isfile(scan_path):
                continue

            scan_file = open(scan_path)
            scan = np.fromfile(scan_file, np.double)
            scan_file.close()

            scan = scan.reshape((len(scan) // 3, 3)).transpose()

            if lidar != 'ldmrs':
                # LMS scans are tuples of (x, y, reflectance)
                reflectance = np.concatenate((reflectance, np.ravel(scan[2, :])))
                scan[2, :] = np.zeros((1, scan.shape[1]))
        else:
            if os.path.isfile(scan_path):
                ptcld = load_velodyne_binary(scan_path)
            else:
                scan_path = os.path.join(lidar_dir, str(timestamps[i]) + '.png')
                if not os.path.isfile(scan_path):
                    continue
                ranges, intensities, angles, approximate_timestamps = load_velodyne_raw(scan_path)
                ptcld = velodyne_raw_to_pointcloud(ranges, intensities, angles)

            reflectance = np.concatenate((reflectance, ptcld[3]))
            scan = ptcld[:3]

        scan = np.dot(np.dot(poses[i], G_posesource_laser), np.vstack([scan, np.ones((1, scan.shape[1]))]))
        pointcloud = np.hstack([pointcloud, scan])

    pointcloud = pointcloud[:, 1:]
    if pointcloud.shape[1] == 0:
        raise IOError("Could not find scan files for given time range in directory " + lidar_dir)

    return pointcloud, reflectance, poses

In [6]:
import argparse
import os
import re
import matplotlib.pyplot as plt
from datetime import datetime as dt
from image import load_image
from camera_model import CameraModel

def play_camera(im_dir = im_dir_left, models = models, start_time=0, end_time=1):
    
    camera = re.search('(stereo|mono_(left|right|rear))', im_dir).group(0)
    timestamps_path = os.path.join(os.path.join(im_dir, os.pardir, camera + '.timestamps'))
    model = CameraModel(models, im_dir)
    current_chunk = 0
    timestamps_file = open(timestamps_path)
    
    i = 0
    IM = []
    for line in timestamps_file:
        tokens = line.split()    
        timestamp = int(tokens[0])
        if start_time <= timestamp <= end_time:
        
            datetime = dt.utcfromtimestamp(timestamp/1000000)
            chunk = int(tokens[1])

            filename = os.path.join(im_dir, tokens[0] + '.png')
            if not os.path.isfile(filename):
                if chunk != current_chunk:
                    print("Chunk " + str(chunk) + " not found")
                    current_chunk = chunk
                continue

            current_chunk = chunk

            img = load_image(filename, model)
            IM.append(img)
            #plt.imshow(img)
            #plt.ylabel(i)
            #plt.xlabel(datetime)
            #plt.xticks([])
            #plt.yticks([])
            #plt.pause(0.01)            
        i += 1
    return np.asarray(IM)

In [20]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import argparse
from build_pointcloud import build_pointcloud
from transform import build_se3_transform
from image import load_image
from camera_model import CameraModel

def project_laser_to_image(image_idx=10, im_dir = im_dir_left, models = models, extrinsics_dir=extrinsics_dir, poses_file=poses_file, lidar_dir=lidar_dir):

    model = CameraModel(models, im_dir)

    extrinsics_path = os.path.join(extrinsics_dir, model.camera + '.txt')
    with open(extrinsics_path) as extrinsics_file:
        extrinsics = [float(x) for x in next(extrinsics_file).split(' ')]

    G_camera_vehicle = build_se3_transform(extrinsics)
    G_camera_posesource = None

    poses_type = re.search('(vo|ins|rtk)\.csv', poses_file).group(1)
    if poses_type in ['ins', 'rtk']:
        with open(os.path.join(extrinsics_dir, 'ins.txt')) as extrinsics_file:
            extrinsics = next(extrinsics_file)
            G_camera_posesource = G_camera_vehicle * build_se3_transform([float(x) for x in extrinsics.split(' ')])
    else:
        # VO frame and vehicle frame are the same
        G_camera_posesource = G_camera_vehicle

    timestamps_path = os.path.join(im_dir, os.pardir, model.camera + '.timestamps')
    if not os.path.isfile(timestamps_path):
        timestamps_path = os.path.join(im_dir, os.pardir, os.pardir, model.camera + '.timestamps')

    timestamp = 0
    with open(timestamps_path) as timestamps_file:
        for i, line in enumerate(timestamps_file):
            if i == image_idx:
                timestamp = int(line.split(' ')[0])

    pointcloud, reflectance = build_pointcloud(lidar_dir, poses_file, extrinsics_dir,
                                               timestamp - 1e7, timestamp + 1e7, timestamp)
    pointcloud = np.dot(G_camera_posesource, pointcloud)
    image_path = os.path.join(im_dir, str(timestamp) + '.png')
    image = load_image(image_path, model)

    uv, depth = model.project(pointcloud, image.shape)

    # plt.imshow(image)
    # plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=2, c=depth, edgecolors='none', cmap='jet')
    # plt.xlim(0, image.shape[1])
    # plt.ylim(image.shape[0], 0)
    # plt.xticks([])
    # plt.yticks([])
    # plt.show()
    
    return image, depth, uv, pointcloud
    
image, depth, uv, pointcloud = project_laser_to_image()

FileNotFoundError: [Errno 2] No such file or directory: '/home/ios/data/oxford_sample/mono_left/../../mono_left.timestamps'

In [8]:
pc = np.asarray(pointcloud.T[:, :3].astype(np.float32))
meshplot.plot(pc, c=pc[:, 1], shading={"point_size": 0.5, "width": 400, "height": 400})

NameError: name 'pointcloud' is not defined

In [9]:
sec = 2
IM = play_camera(start_time=1418381798113072, end_time=1418381798113072 + sec * 1000 * 1000)
IM.shape

FileNotFoundError: [Errno 2] No such file or directory: '/home/ios/data/oxford_sample/mono_left/../mono_left.timestamps'

In [10]:
def Lidar_to_Mono(extr_mono_path=extrinsics_dir + "/mono_left.txt", extr_lidar_path=extrinsics_dir + "/ldmrs.txt"):   
    extrinsics_path = extr_mono_path
    with open(extrinsics_path) as extrinsics_file:
        extrinsics = [float(x) for x in next(extrinsics_file).split(' ')]
    G_camera_vehicle = build_se3_transform(extrinsics)
    
    extrinsics_path = extr_lidar_path
    with open(extrinsics_path) as extrinsics_file:
        extrinsics = [float(x) for x in next(extrinsics_file).split(' ')]
    G_lidar_vehicle = build_se3_transform(extrinsics)
    
    T = G_camera_vehicle[:3, 3] - G_lidar_vehicle[:3, 3]
    R = G_camera_vehicle[:3, :3] @  G_lidar_vehicle[:3, :3].T
    
    SE = np.zeros((4, 4))
    SE[:3, :3] = np.array(R)
    SE[:3, 3] = np.array(T).flatten()
    SE[3, 3] = 1
    
    return SE

Lidar_to_Mono(extrinsics_dir + "/mono_left.txt"), Lidar_to_Mono(extrinsics_dir + "/mono_right.txt")

(array([[ 0.30164739, -0.95046503,  0.07500055, -1.6254    ],
        [ 0.90174962,  0.25887064, -0.34616992,  1.6285    ],
        [ 0.30960696,  0.17205297,  0.93516913, -1.0974    ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.32721212,  0.93838189,  0.1112279 , -1.7936    ],
        [-0.89269826,  0.26837006,  0.36203223, -1.69      ],
        [ 0.30987425, -0.21775429,  0.92550582, -1.0551    ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [17]:
sec = 1
pointcloud, reflectance, poses = build_pointcloud(lidar_dir, poses_file, extrinsics_dir, 1418381798113072, 1418381798113072 + sec * 1000 * 1000, origin_time=-1)
pointcloud.shape

FileNotFoundError: [Errno 2] No such file or directory: '/home/ios/data/oxford_sample/ldmrs/../ldmrs.timestamps'

In [ ]:
for i in range(5):
    print( poses[i][:, 3] ) # Ts 

In [16]:
pc = np.asarray(pointcloud.T[:, :3].astype(np.float32))
meshplot.plot(pc, c=pc[:, 1], shading={"point_size": 0.5, "width": 800, "height": 800})

NameError: name 'pointcloud' is not defined

In [13]:
from sklearn.neighbors import NearestNeighbors

def get_iis_keypoints(pc):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(pc)
    keypoints = open3d.geometry.keypoint.compute_iss_keypoints(pcd,
                                                            salient_radius=0.03,
                                                            non_max_radius=0.03,
                                                            gamma_21=0.5,
                                                            gamma_32=0.5)
    keypoints = np.asarray(keypoints.points)
    keypoints.shape
    return keypoints


def get_volumes(keypoints, pc, radius=1):
    rnr = NearestNeighbors(radius=1)
    rnr.fit(pc)
    points, idx = rnr.radius_neighbors(keypoints)
    volumes = []
    for i in range(len(idx)):
        points = pc[np.asarray(idx[i])]
        points -= points.mean(0)
        volumes.append(points)
    return volumes

def get_sift_keypoints(im, sift=cv2.SIFT_create()):
    assert im.shape[2] == 3

    def unpackSIFTOctave(kpt):
        """unpackSIFTOctave(kpt)->(octave,layer,scale)
        @created by Silencer at 2018.01.23 11:12:30 CST
        @brief Unpack Sift Keypoint by Silencer
        @param kpt: cv2.KeyPoint (of SIFT)
        """
        _octave = kpt.octave
        octave = _octave&0xFF
        layer  = (_octave>>8)&0xFF
        if octave>=128:
            octave |= -128
        if octave>=0:
            scale = float(1/(1<<octave))
        else:
            scale = float(1<<-octave)
        return (octave, layer, scale)
    
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    kp = sift.detect(gray, None)
    return [(*unpackSIFTOctave(kpt), int(kpt.pt[0]),int(kpt.pt[1]))  for kpt in kp]
    

def get_patches(keypoints, IM, basic_shape=(256, 256)):
    patches = []
    for kp in keypoints:
        octave, layer, scale, x, y = kp 
        shp = int(basic_shape[0] * scale // 2)
        patch = IM[y-shp:y+shp,x-shp:x+shp]    
        if len(patch.flatten()) > 0:
            patches.append(patch)
    return patches    


keypoints = get_iis_keypoints(pc)
volumes = get_volumes(keypoints, pc)
kp = get_sift_keypoints(IM[0], sift=cv2.SIFT_create())
patches = get_patches(kp, IM[0])

NameError: name 'pc' is not defined

In [ ]:
plt.imshow(patches[12])

In [15]:
meshplot.plot(volumes[0], c=volumes[0][:, 1], shading={"point_size": 0.5, "width": 300, "height": 300})

NameError: name 'volumes' is not defined